In [1]:
print("hej")

hej


In [2]:
from pytradfri import Gateway
from pytradfri.device import light_control, light
from pytradfri.util import load_json, save_json
from pytradfri.command import Command
from pytradfri.device.light import Light
import uuid
from typing import List
import asyncio
from tqdm import tqdm
import time
import multiprocessing
import atexit
from client import get_api, get_api_async
import  math
import concurrent.futures

In [3]:

GATEWAY_IP = "192.168.1.175"

CONFIG_FILE = "tradfri_standalone_psk.conf"

In [4]:
api = await get_api_async(GATEWAY_IP)
api

<bound method APIFactory.request of <pytradfri.api.aiocoap_api.APIFactory object at 0x796211b26190>>

In [5]:
light_ids = [
  65540, 65541, 65542,
  65543, 65544, 65545,
  65546, 65547, 65553,
  65552, 65551, 65550,
  65549
]

In [7]:
''
gateway = Gateway()
gateway_info = await api(Gateway.get_gateway_info())

# Print the firmware version
print("Gateway Firmware Version:", gateway_info.firmware_version)

Gateway Firmware Version: 1.21.31


In [8]:
devices_command = gateway.get_devices()
devices_commands = await api(devices_command)
devices = await api(devices_commands)
lights = [dev for dev in devices if dev.has_light_control]
lights

[<65536 - TRADFRI bulb 11 (TRADFRI bulb E14 CWS 470lm)>,
 <65540 - TRADFRI bulb 5 (TRADFRI bulb E14 CWS 470lm)>,
 <65537 - TRADFRI bulb (TRADFRI bulb E14 CWS 470lm)>,
 <65550 - TRADFRI bulb 2 (TRADFRI bulb E14 CWS 470lm)>,
 <65542 - TRADFRI bulb 7 (TRADFRI bulb E14 CWS 470lm)>,
 <65552 - Bulb 13 (TRADFRI bulb E14 CWS 470lm)>,
 <65539 - TRADFRI bulb 3 (TRADFRI bulb E14 CWS 470lm)>,
 <65538 - TRADFRI bulb 4 (TRADFRI bulb E14 CWS 470lm)>,
 <65549 - TRADFRI bulb 10 (TRADFRI bulb E14 CWS 470lm)>,
 <65544 - TRADFRI bulb 9 (TRADFRI bulb E14 CWS 470lm)>,
 <65551 - TRADFRI bulb 12 (TRADFRI bulb E14 CWS 470lm)>,
 <65545 - TRADFRI bulb 8 (TRADFRI bulb E14 CWS 470lm)>,
 <65543 - TRADFRI bulb 6 (TRADFRI bulb E14 CWS 470lm)>]

In [9]:
len(lights), len(light_ids), [d for d in devices if not d.has_light_control]

(13, 13, [])

In [10]:
print(light_control.LightControl(lights[5]))

<LightControl for Bulb 13 (1 devices)>


In [11]:
async def exe_time(func):
    start = time.time
    return func(), time.time - start


In [12]:

time.time()

1713191975.8595898

In [13]:
12%10

2

In [14]:
async def blink0(light, up_time, down_time, min_dim = 10, max_dim = 100, start_time = None,
                tolerance = 10):
    start_time = start_time if start_time != None else time.time()
   
    a = None
    b = None
    misses = []
    while True:
     

        
        dif1 =  (start_time + down_time) - time.time()
        await asyncio.sleep(max(dif1, 0))
        if a != None:
            await a
        a = api(light_control.LightControl(light).set_dimmer(min_dim))
        
        dif2 = (start_time + down_time + up_time) - time.time()
        await asyncio.sleep(max(dif2, 0))
        if b != None:
            await b
        b = api(light_control.LightControl(light).set_dimmer(max_dim))
        start_time = start_time +  down_time + up_time
        if dif1 == 0 or dif2 == 0:
            misses.append(dif1+dif2)
        else:
            misses = []
        if len(misses) > tolerance:
            print(misses)
            print("To many misses")
            await api(light_control.LightControl(light).set_dimmer(0))
            return

In [ ]:
tasks = [blink0(l, 2, 2, 10, 100) for l in lights]
await asyncio.gather(*tasks)

An exception that is not an aiocoap Error was raised from a transport; please report this as a bug in aiocoap: CredentialsMissingError('No suitable credentials for coaps://192.168.1.175/15001/65536')
An exception that is not an aiocoap Error was raised from a transport; please report this as a bug in aiocoap: CredentialsMissingError('No suitable credentials for coaps://192.168.1.175/15001/65540')
An exception that is not an aiocoap Error was raised from a transport; please report this as a bug in aiocoap: CredentialsMissingError('No suitable credentials for coaps://192.168.1.175/15001/65537')
An exception that is not an aiocoap Error was raised from a transport; please report this as a bug in aiocoap: CredentialsMissingError('No suitable credentials for coaps://192.168.1.175/15001/65550')
An exception that is not an aiocoap Error was raised from a transport; please report this as a bug in aiocoap: CredentialsMissingError('No suitable credentials for coaps://192.168.1.175/15001/65542')


ServerError: ('There was an error with the request.', CredentialsMissingError('No suitable credentials for coaps://192.168.1.175/15001/65542'))

In [ ]:

async def blink(light, next_on_beat, next_off_beat, min_dim = 10, max_dim = 100):
   
    a = None
    b = None
    while True:
        if a != None:
            await a
        await next_on_beat().wait()
        a = api(light_control.LightControl(light).set_dimmer(min_dim))
     
        if b != None:
            await b
        await next_off_beat().wait()
        b = api(light_control.LightControl(light).set_dimmer(max_dim))
        
       
       
    

In [ ]:
class Syncer:
    def __init__(self, stop) -> None:
        
        self.on_beat = asyncio.Event()
        self.on_beat.clear()
        self.off_beat = asyncio.Event()
        self.off_beat.clear()
        self.stop = stop    
    async def tick(self):
        while not self.stop():
            self.on_beat.set()
            await asyncio.sleep(0.5)
            self.on_beat.clear()
            self.off_beat.set()
            await asyncio.sleep(0.5)
            self.off_beat.clear()

In [ ]:
tasks = []
tasks = [tick()] + [blink(light, on_beat, off_beat, 2, 130) for light in ]
await asyncio.gather(*tasks)    


CancelledError: 

In [ ]:
tasks = []
for light in lights[::-1]:
    time.sleep(0.0)
    
    tasks.append(api(light_control.LightControl(light).set_dimmer(0)))
    tasks.append(api(light_control.LightControl(light).set_hex_color("ffffff")))

await asyncio.gather(*tasks)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [36]:
lights[11].id

65545

In [26]:
light_control.LightControl(lights[11]).set_dimmer(2).data

{'3311': [{'5851': 2}]}

In [17]:

b = api(light_control.LightControl(lights[11]).set_dimmer(2))
await b


In [ ]:
lights[12]

<65545 - TRADFRI bulb 8 (TRADFRI bulb E14 CWS 470lm)>

65547

In [ ]:
lights[9].id

65550

In [ ]:
[l.id for l in lights]

[65536,
 65537,
 65538,
 65539,
 65540,
 65542,
 65544,
 65545,
 65549,
 65550,
 65543,
 65551,
 65552]